In [18]:
import pandas as pd
pd.set_option("display.precision", 2)

In [19]:
file_transactions = 'transactions.csv'

#

# 1. Read transactions history

In [20]:
df_trans = pd.read_csv(file_transactions)
df_trans.tail()

,Date,Ticker,Action,Quantity,Price,Account
1,2022-01-13,ASML,Buy,20,730,XR
2,2022-01-13,AAL,Buy,100,19,YT
3,2022-01-14,ASML,Sell,5,740,YR
4,2022-01-15,ASML,Buy,10,745,YT
5,2022-01-15,ASML,Sell,10,750,YT


# Create Positions

In [21]:
# Add Amount column for each transaxtion
df_trans.sort_values(['Ticker','Date','Action'], ascending=[True, True, True], inplace=True)
df_trans['Amount'] = df_trans['Quantity'] * df_trans['Price']
df_trans

,Date,Ticker,Action,Quantity,Price,Account,Amount
2,2022-01-13,AAL,Buy,100,19,YT,1900
1,2022-01-13,ASML,Buy,20,730,XR,14600
3,2022-01-14,ASML,Sell,5,740,YR,3700
4,2022-01-15,ASML,Buy,10,745,YT,7450
5,2022-01-15,ASML,Sell,10,750,YT,7500
0,2022-01-01,CASH,Deposit,1,10000,XT,10000


In [22]:
# Buy a stock, then we have more Quantity(+) and Amount(+), otherwise we have less (-)
df = df_trans[df_trans['Ticker'] != 'CASH'].copy(deep=True)
df['Signed Quantity'] = df.apply(lambda x: (1 if (x.Action == 'Buy' or x.Action == 'Deposit') else -1) * x.Quantity, axis=1)
df['Signed Amount'] = df.apply(lambda x: (1 if (x.Action == 'Buy' or x.Action == 'Deposit') else -1) * x.Amount, axis=1)
df

,Date,Ticker,Action,Quantity,Price,Account,Amount,Signed Quantity,Signed Amount
2,2022-01-13,AAL,Buy,100,19,YT,1900,100,1900
1,2022-01-13,ASML,Buy,20,730,XR,14600,20,14600
3,2022-01-14,ASML,Sell,5,740,YR,3700,-5,-3700
4,2022-01-15,ASML,Buy,10,745,YT,7450,10,7450
5,2022-01-15,ASML,Sell,10,750,YT,7500,-10,-7500


In [23]:
# Sum signed quantity and signed amount
df_ahp = df.groupby(['Ticker'])[['Signed Quantity', 'Signed Amount']].sum().reset_index()
df_ahp

,Ticker,Signed Quantity,Signed Amount
0,AAL,100,1900
1,ASML,15,10850


In [24]:
# Average Holding Price = Signed Amount / Signed Quantity
df_ahp['Average Holding Price'] = df_ahp['Signed Amount'] / df_ahp['Signed Quantity']
df_ahp

,Ticker,Signed Quantity,Signed Amount,Average Holding Price
0,AAL,100,1900,19.00
1,ASML,15,10850,723.33


In [28]:
import pandas_datareader as pdr
import datetime as dt

end = dt.date.today()
start = end - dt.timedelta(days=1)

ticker = 'ASML'
df = pdr.get_data_yahoo(ticker, start, end)
df['Close']

Date
2022-01-21    694.73
Name: Close, dtype: float64